<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

Загрузим нужные библиотеки

In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.pipeline import Pipeline
nltk.download('wordnet','stopwords','punkt','averaged_perceptron_tagger')

True

Прочитаем файл /datasets/toxic_comments.csv и сохраним его в переменную 'comment'.

In [2]:
# читаем файл
comment = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

Напишем функцию, которая выводит общую информацию о таблице, а так же посчитает пропуски и проверит на наличие дубликатов.

In [3]:
#пишем функцию summary
def summary(df):
    print(df.info())
    display(df.head())
    count=0
    for col in df.columns:
        if df.isna().sum()[col]==0:
            count+=1
        else:
            print(f'Пропущенных значений в столбце {col}: {df.isna().sum()[col]}')
    if count==len(df.columns):
        print('Пропущенных значений в таблице нет')
    print(f'Повторов в таблице {df.duplicated().sum()}') 
    display(df.describe(percentiles=[.25,.5]))

In [4]:
# смотрим на данные таблицы
summary(comment)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Пропущенных значений в таблице нет
Повторов в таблице 0


,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
max,1.000000


Всего имеется 159292 комментарий. Данные без пропусков и дубликатов. Так как перед нами стоит задача классификации, то проверим веса целевого признака.

In [5]:
# исследуем веса целевого признака
display(comment['toxic'].value_counts())
class_frequency = comment['toxic'].value_counts()[0] / comment['toxic'].value_counts()[1]
display(class_frequency)

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

Веса не сбалансированны. Соотношение почти 9:1. При обучениее моделей, будем использовать изменение весов классов.

Для дальнейшей работы проведём лематизацию и очистку от лишних символов с помощью регулярных выражений.

In [6]:
#выполним токенизациию и лемматизацию массива текстов
L = WordNetLemmatizer()
def lemmatizered(comment):
    comment_new = []
    for sentence in comment:
        word_list = nltk.word_tokenize(sentence)
        comment_new.append(' '.join([L.lemmatize(w) for w in word_list]))
    return comment_new

In [7]:
#возвращаем значение часть речи «part-of-speech» (POS-тег).
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
 
    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
# выполним токенезацию и лемматизацию с учетом POS-тег
def get_word_text(comment):
    comment_new = []
    for sentence in comment:
        comment_new.append(' '.join([L.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence) if not w in stopwords]))
    return comment_new

In [9]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
 %%time
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
comment['lemm_text'] = get_word_text(comment['text'])

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


CPU times: user 13min 42s, sys: 1min 11s, total: 14min 54s
Wall time: 14min 55s


In [11]:
comment = comment.drop(['text'], axis=1)

Подготовим данные для обучения моделей. Зададим признак и целевой признак. Разобъём исходный датасет на 3 выборки в пропорции 60/20/20(т.к. исходный датасет большого объёма). импортируем стоп-слова. Произведём векторизацию слов.

In [12]:
# задаём признаки и целевой признак
target = comment['toxic']
features = comment.drop(['toxic'], axis=1)

In [13]:
# разбиваем исходный датасет на выборки
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=12345)

In [14]:
# загружаем стоп-слова
# создаём счетчик, указав в нём стоп-слова
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# посчитаем tf-idf  для признаков
features_train = count_tf_idf.fit_transform(features_train['lemm_text'])
features_valid = count_tf_idf.transform(features_valid['lemm_text'])
features_test = count_tf_idf.transform(features_test['lemm_text'])
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(95575, 131678)
(31858, 131678)
(31859, 131678)


**Вывод**
* были загружены данные для обучения модели
* был исследован баланс классов целевого признака
* принято решение о борьбе с дисбаланосм с помощью взвешивания классов
* была произведена лемматизация и удалены исходные твиты
* были заданы признаки и целевой признак
* произведена векторизация признаков
* данные разбиты на выборки

## Обучение

Для обучения выберем следующие модели:

 * LogisticRegression
 * DecisionTreeClassifier
 * CatBoostClassifier

### Логистическая регрессия

Найдём гиперпараметры для логистической регрессии

In [16]:
dict_classes={0:1, 1:class_frequency}

In [17]:
%%time

model_lr = LogisticRegression()
hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10],
                'class_weight':[dict_classes]}]
clf = GridSearchCV(model_lr, hyperparams, scoring='f1',cv=4)
clf.fit(features_train, target_train)
print("лучшие параметры для регрессии:")
print()
lr_best_params = clf.best_params_
print(lr_best_params)

лучшие параметры для регрессии:

{'C': 10, 'class_weight': {0: 1, 1: 8.841344371679229}, 'solver': 'lbfgs'}
CPU times: user 6min 43s, sys: 7min 54s, total: 14min 37s
Wall time: 14min 38s


In [18]:
%%time

model_lr = LogisticRegression()
model_lr.set_params(**lr_best_params)
model_lr.fit(features_train, target_train)
target_predict = model_lr.predict(features_valid)
valid_f1_lr = f1_score(target_valid, target_predict)
print('F1 на валидации', valid_f1_lr)

F1 на валидации 0.7560097883978696
CPU times: user 19.6 s, sys: 23.4 s, total: 43 s
Wall time: 43.1 s


### Решающее дерево

In [19]:
%%time

model_dtс = DecisionTreeClassifier()
hyperparams = [{'max_depth':[x for x in range(9,45,2)],
                'random_state':[12345],
                'class_weight':[dict_classes]}]

clf = GridSearchCV(model_dtс , hyperparams, scoring='f1',cv=4)
clf.fit(features_train, target_train)
print("Лучшие параметры для решающего дерева:")
print()
dtc_best_params = clf.best_params_
print(dtc_best_params)

Лучшие параметры для решающего дерева:

{'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 35, 'random_state': 12345}
CPU times: user 19min 8s, sys: 11.3 s, total: 19min 20s
Wall time: 19min 20s


In [20]:
%%time

model_dtc = DecisionTreeClassifier()
model_dtc.set_params(**dtc_best_params)
model_dtc.fit(features_train, target_train)
target_predict = model_dtc.predict(features_valid)
valid_f1_dtc = f1_score(target_valid, target_predict)
print('F1 для решающего дерева', valid_f1_dtc)

F1 для решающего дерева 0.6268364348677767
CPU times: user 22 s, sys: 140 ms, total: 22.2 s
Wall time: 22.2 s


### CatBoostClassifier

In [21]:
%%time

model_cbc = CatBoostClassifier(verbose=False, iterations=200)
model_cbc.fit(features_train, target_train)
target_predict_cbc = model_cbc.predict(features_valid)
cv_f1_cbc = cross_val_score(model_cbc,
                                         features_train, 
                                         target_train, 
                                         cv=4, 
                                         scoring='f1').mean()
valid_f1_cbc = f1_score(target_valid, target_predict_cbc)
print('F1 для CatBoostClassifier', valid_f1_cbc)

F1 для CatBoostClassifier 0.7380696088984572
CPU times: user 25min 28s, sys: 6min 27s, total: 31min 55s
Wall time: 31min 59s


**Вывод**
* были обучены 3 модели
* по итогам обучения метрика F-1 соответсвует требованиям у модели логистической регрессии

## Тестирование

Проведём тестирование моделей LogisticRegression

In [22]:
# находим F-1 меру тестовой выборки
decision_model_lr = LogisticRegression(**lr_best_params)
decision_model_lr.fit(features_train, target_train)
test_predictions_lr = decision_model_lr.predict(features_test)
f1_score_decision_modelt_test_lr = f1_score(target_test, test_predictions_lr)
display('F-1 модели LogisticRegression:', f1_score_decision_modelt_test_lr)

'F-1 модели LogisticRegression:'

0.7473668595163923

## Выводы

* В ходе исселодования была найдена наилучшая модель для определения токсичности твитов - модель LogisticRegression